In [108]:
import json

import pandas as pd
import numpy as np
import networkx as nx

import torch
from torch_geometric.nn import GATv2Conv, global_max_pool
from torch_geometric.data import Data
from torch_geometric.loader import GraphSAINTRandomWalkSampler, GraphSAINTNodeSampler
from matplotlib import pyplot as plt

from train_utils import *
from product_graph import *
from tqdm.notebook import tqdm
from torch_geometric.utils import to_dense_adj
from torch.nn import MSELoss
from product_graph import generate_parametric_product_graph
import networkx as nx
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.utils import from_scipy_sparse_matrix

In [109]:
class GATv3Conv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, concat = True, heads=1) -> None:
        super().__init__()
        self.beta = torch.nn.Parameter(torch.tensor(0.5))
        self.conv = GATv2Conv(in_channels, out_channels, heads, concat, add_self_loops=False)

    def forward(self, x, edge_index, edge_weights):
        H, C = self.conv.heads, self.conv.out_channels

        if isinstance(x, torch.Tensor):
            assert x.dim() == 2
            x_l = self.conv.lin_l(x).view(-1, H, C)
            if self.conv.share_weights:
                x_r = x_l
            else:
                x_r = self.conv.lin_r(x).view(-1, H, C)
        else:
            raise TypeError("x must be a Tensor")

        assert x_l is not None
        assert x_r is not None

        # edge_updater_type: (x: PairTensor, edge_attr: OptTensor)
        alpha = self.conv.edge_updater(edge_index, x=(x_l, x_r), edge_attr=None)
        
        alpha = (1-self.beta) * alpha + self.beta * edge_weights.view(edge_weights.shape[0],1)
        # propagate_type: (x: PairTensor, alpha: Tensor)
        out = self.conv.propagate(edge_index, x=(x_l, x_r), alpha=alpha)

        if self.conv.concat:
            out = out.view(-1, self.conv.heads * self.conv.out_channels)
        else:
            out = out.mean(dim=1)

        if self.conv.bias is not None:
            out = out + self.conv.bias

        return out
    

class GATNN(torch.nn.Module):
    def __init__(self, in_dim, hidden_size, out_dim, in_head=8, out_head=1, p=0.25) -> None:
        super().__init__()
        self.hid = hidden_size
        self.in_head = in_head
        self.out_head = out_head
        self.p = p
        
        self.conv1 = GATv3Conv(in_channels=in_dim, 
                               out_channels=self.hid, 
                               heads=self.in_head)
        
        self.conv2 = GATv3Conv(in_channels=self.hid*self.in_head, 
                               out_channels=self.hid, 
                               heads=self.out_head, 
                               concat=False)
        
        self.lin = nn.Linear(self.hid, out_dim)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=self.p, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        
        batch = torch.zeros(x.size(0), dtype=torch.long)
        x = global_max_pool(x, batch, size=x.shape[0]//4)

        x = self.lin(x)
        return x

In [110]:
dynamic_data = torch.tensor(np.load("data/preprocessed/dynamic_data.npy", allow_pickle=True))
S = torch.tensor(np.load("data/adjacency/coords_features.npy", allow_pickle=False))

calendar_df = pd.read_csv('data/preprocessed/calendar.csv')
data = Data(x = dynamic_data)
edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()
edge_weight = S[edge_index[0], edge_index[1]]
data.edge_index = edge_index
data.edge_weight = edge_weight

/var/folders/k7/vcjbn_bd7wdb2g9h92g_pkj80000gn/T/ipykernel_1872/4039232763.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()


In [105]:
criterion = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATNN(in_dim=1, hidden_size=32, out_dim=1).cpu()

loader = GraphSAINTNodeSampler (
    data,
    batch_size=200,
    num_steps=6,
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)

temporal_adj = np.array([[0, 0, 0, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

AttributeError: 'dict' object has no attribute 'edge_index'

In [33]:
totalCount = 0

for param in model.parameters():
    totalCount += param.nelement()

totalCount

1451

In [43]:
train_loss = []
for epoch in range(20):
    model.train()
    total_loss = 0
    for i, batch in enumerate(loader):
        batch_loss = 0
        dataset = create_forecasting_dataset(batch.x.T,
                                        splits = None,
                                        pred_horizen= 1,
                                        obs_window= 4,
                                        verbose = 0)
        batch_sample = Data(x = torch.tensor(dataset['trn']['data']), y = torch.tensor(dataset['trn']['labels']), 
                            edge_index= batch.edge_index, edge_weight = batch.edge_weight)

        batch_adj = to_dense_adj(batch_sample.edge_index, edge_attr=batch_sample.edge_weight).squeeze(dim = 0).numpy()
        
        product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
        product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
        
        batch_x = batch_sample.x.reshape(batch_sample.x.shape[0], batch_sample.x.shape[1]* batch_sample.x.shape[2], 1)
        batch_y = batch_sample.y.reshape(batch_sample.y.shape[0], batch_sample.y.shape[1])
        
        for i in tqdm(range(batch_sample.x.shape[0])):
            optimizer.zero_grad()
            out = model(batch_x[i].float(), product_edge_index,product_edge_weight.float())
            # print(f'out shape: {out.shape} \n y shape {batch_y[i].shape}')
            # print(out)
            loss = criterion(out, batch_y[i].float())
            batch_loss += loss
        
            loss.backward()
            optimizer.step()
            
        total_loss += batch_loss
        print(f'Epoch: {epoch} Batch Loss: {batch_loss}')
    train_loss.append(total_loss)

  0%|          | 0/275 [00:00<?, ?it/s]

c:\Users\erena\anaconda3\envs\Networks\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([191])) that is different to the input size (torch.Size([191, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0 Batch Loss: 33260036.0


  0%|          | 0/275 [00:00<?, ?it/s]

Epoch: 0 Batch Loss: 16121438.0


...

Epoch: 1 Batch Loss: 18469132.0


  0%|          | 0/275 [00:00<?, ?it/s]

Epoch: 1 Batch Loss: 17200608.0


  0%|          | 0/275 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [111]:
data = create_forecasting_dataset(dynamic_data.T,
                                      splits = [0.8, 0.1, 0.1],
                                      pred_horizen= 1,
                                      obs_window= 4,
                                      verbose = 0)
edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()
edge_weight = S[edge_index[0], edge_index[1]]

criterion = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATNN(in_dim=1, hidden_size=32, out_dim=1).cpu()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)

temporal_adj = np.array([[0, 0, 0, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

/var/folders/k7/vcjbn_bd7wdb2g9h92g_pkj80000gn/T/ipykernel_1872/485915226.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()


In [112]:

# loop over the number of samples
train_samples = data['trn']['data'].shape[0]
val_samples = data['val']['data'].shape[0]
train_losses = []
val_losses = []
for epoch in range(20):
    total_loss = 0
    for i in range(train_samples):
        # Create a torch geometric data over each graph 
        outer_batch = Data(x = torch.tensor(data['trn']['data'][i]), y = torch.tensor(data['trn']['labels'][i].squeeze()),
                        edge_index=edge_index, edge_weight = edge_weight) 
        
        train_loader = GraphSAINTNodeSampler(outer_batch, batch_size=100, num_steps=6)
        for inner_batch in train_loader:
            model.train()
            batch_loss = 0
            batch_adj = to_dense_adj(inner_batch.edge_index, edge_attr=inner_batch.edge_weight).squeeze(dim = 0)
            
            batch_adj = batch_adj.numpy()
            
            product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
            product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
            
            
            batch_x =inner_batch.x.reshape(inner_batch.x.shape[0]* inner_batch.x.shape[1], 1)
            
            batch_y = inner_batch.y.unsqueeze(dim = 1)
            optimizer.zero_grad()
            out = model(batch_x.float(), product_edge_index,product_edge_weight.float())
            print(out.shape)
            print(batch_y.shape)
            loss = criterion(out, batch_y.float())
            batch_loss += loss
        
            loss.backward()
            optimizer.step()
            total_loss += batch_loss
        
    train_losses.append(total_loss)
    val_loss = 0
    for i in range(val_samples):
        model.eval()
        # Create a torch geometric data over each graph 
        outer_batch = Data(x = torch.tensor(data['val']['data'][i]), y = torch.tensor(data['val']['labels'][i].squeeze()),
                        edge_index=edge_index, edge_weight = edge_weight) 
        
        val_loader = GraphSAINTNodeSampler(outer_batch, batch_size=200, num_steps=6)
        for inner_batch in val_loader:
            with torch.no_grad:
                batch_loss = 0
                batch_adj = to_dense_adj(inner_batch.edge_index, edge_attr=inner_batch.edge_weight).squeeze(dim = 0)
                
                batch_adj = batch_adj.numpy()
                
                product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
                product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
                
                
                batch_x =inner_batch.x.reshape(inner_batch.x.shape[0]* inner_batch.x.shape[1], 1)
                
                batch_y = inner_batch.y.unsqueeze(dim = 1)
                out = model(batch_x.float(), product_edge_index,product_edge_weight.float())

                loss = criterion(out, batch_y.float())

                val_loss += batch_loss
        
    val_losses.append(val_loss)
        
        


torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([99, 1])
torch.Size([99, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([95, 1])
torch.Size([95, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([96, 1])
torch.Size([96, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([96, 1])
torch.Size([96, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([97, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([98, 1])
torch.Size([99, 1])
torch.Size([99, 1])


KeyboardInterrupt: 